In [3]:
import cv2
#from google.colab.patches import cv2_imshow
import os
import numpy as np
import pandas as pd
import math
from PIL import Image
import PIL
import numpy as np
from keras.preprocessing import image
from keras.models import Model,load_model
from keras.applications.imagenet_utils import preprocess_input,decode_predictions
from keras.applications.resnet50 import ResNet50
#from keras.applications.vgg16 import VGG16
from tensorflow.keras import backend as K
from matplotlib import pyplot as plt
from elapsedtimer import ElapsedTimer
import fire
import ffmpeg
import subprocess
import shutil
import glob
import tqdm
from tqdm import tqdm

In [4]:
class VideoCaptioningPreProcessing:
   # print("test")
    def __init__(self,video_dest,feat_dir,temp_dest,img_dim=224,channels=3,batch_size=1,frames_step=80):
    
        self.img_dim = img_dim
        self.channels = channels
        self.video_dest = video_dest
        self.feat_dir = feat_dir
        self.temp_dest = temp_dest
        self.batch_cnn = batch_size
        self.frames_step = frames_step
    def video_to_frames(self,video):
        #with open(os.devnull, "w") as ffmpeg_log:
        if os.path.exists(self.temp_dest):
            print(" cleanup: " + self.temp_dest + "/")
            shutil.rmtree(self.temp_dest)
        os.mkdir(self.temp_dest)
#                 video_to_frames_cmd = ["ffmpeg",'-i', video,'-vf', "scale=-1:300", 
#                                            '-qscale:v', "2", 
#                                            '{0}/%06d.jpg'.format(self.temp_dest)]
#                 subprocess.call(video_to_frames_cmd,stdout=ffmpeg_log, stderr=ffmpeg_log)
        #video_path=os.path.join(source_folder, filename)
        #os.chdir(self.temp_dest) 
        cap = cv2.VideoCapture(video)
        count=0
        filename= r"{}".format(video)
        nfilename = os.path.normpath(filename)
        vfilename=nfilename.split('\\').pop()
        vfilename=vfilename.split('.')[0]
        #print(video)
        while (cap.isOpened()):
            count+=1
            current_frame=cap.get(1)# current frame number
            #print(current_frame)
            ret,frame=cap.read()
            #print(ret,count)
            sfile=self.temp_dest+'\\'+f'{vfilename}_frame{int(current_frame)}.jpg'
            #print(sfile)
            cv2.imwrite(sfile,frame)
            if (ret!=True) or (count==self.frames_step):
                 break
            #cv2.imwrite(f'{vfilename}_frame{int(current_frame)}.jpg',frame)
            
        #print(i)

        cap.release()  

    def model_cnn_load(self):
        #model = VGG16(weights = "imagenet", include_top=True,input_shape=(self.img_dim,self.img_dim,self.channels))
        model = ResNet50(weights = "imagenet", include_top=True,input_shape=(self.img_dim,self.img_dim,self.channels))
        out = model.layers[-2].output
        model_final = Model(inputs=model.input,outputs=out)
        return model_final

    def load_image(self,path):
        print(path)
        img = cv2.imread(path)
        dimensions=img.shape
        height=img.shape[0]
        width=img.shape[1]
        img = cv2.resize(img,(self.img_dim,self.img_dim))
        return img 

    def extract_feats_pretrained_cnn(self):# Extract the features from the pre-trained CNN 
        #self.img_dim = img_dim
        #self.channels = channels
        #self.video_dest = video_dest
        #self.feat_dir = feat_dir
        #self.temp_dest = temp_dest
        #self.batch_cnn = batch_size
        #self.frames_step = frames_step

        model = self.model_cnn_load()
        print('Model loaded')
        #print(os. getcwd())
        if not os.path.exists(self.feat_dir):
            os.mkdir(self.feat_dir)
            #print("save video feats to %s" % (self.dir_feat))
        video_list = glob.glob(os.path.join(self.video_dest,'*.avi'))
        #print(video_list) 

        for video in video_list:

           # video_id = video.split(".")[0]
            
            filename= r"{}".format(video)
            nfilename = os.path.normpath(filename)
            vfilename=nfilename.split('\\').pop()
            video_id=vfilename.split('.')[0]
            #self.dest = 'cnn_feat' + '_' + video_id
            print(video_id)
            self.video_to_frames(video)

            image_list = sorted(glob.glob(os.path.join(self.temp_dest, '*.jpg')))
            samples = np.round(np.linspace(0, len(image_list) - 1,self.frames_step))
            image_list = [image_list[int(sample)] for sample in samples]
            images = np.zeros((len(image_list),self.img_dim,self.img_dim,self.channels))
            for i in tqdm(range(len(image_list))):
                #print(i)
                img = self.load_image(image_list[i])
                images[i] = img
                images = np.array(images)
                fc_feats = model.predict(images,batch_size=self.batch_cnn)
                img_feats = np.array(fc_feats)
                #print(img_feats[0])
                outfile = os.path.join(self.feat_dir, video_id + '.npy')
                np.save(outfile, img_feats)
                # cleanup
                #shutil.rmtree(self.temp_dest)
                
    def process_main(self):
        self.extract_feats_pretrained_cnn()

In [5]:
source_folder='video_captioning_dataset\YouTubeClips\dest'
#video='video_captioning_dataset\YouTubeClips\dest\mv89psg6zh4_33_46.avi'
feat_dir='video_captioning_dataset\YouTubeClips\dir_feat'
temp='video_captioning_dataset\YouTubeClips\dir_temp'
test=VideoCaptioningPreProcessing(source_folder,feat_dir,temp)
VideoCaptioningPreProcessing.extract_feats_pretrained_cnn(test)
#test.video_to_frames(video)

Model loaded
0xx13BuvVmo_25_36
 cleanup: video_captioning_dataset\YouTubeClips\dir_temp/


  0%|                                                                                           | 0/80 [00:00<?, ?it/s]

video_captioning_dataset\YouTubeClips\dir_temp\0xx13BuvVmo_25_36_frame0.jpg


  1%|█                                                                                  | 1/80 [00:18<24:49, 18.86s/it]

video_captioning_dataset\YouTubeClips\dir_temp\0xx13BuvVmo_25_36_frame1.jpg


...

 98%|███████████████████████████████████████████████████████████████████████████████▉  | 78/80 [21:25<00:34, 17.30s/it]

video_captioning_dataset\YouTubeClips\dir_temp\1dA1RQgPN3c_72_82_frame8.jpg


 99%|████████████████████████████████████████████████████████████████████████████████▉ | 79/80 [21:41<00:17, 17.01s/it]

video_captioning_dataset\YouTubeClips\dir_temp\1dA1RQgPN3c_72_82_frame9.jpg


100%|██████████████████████████████████████████████████████████████████████████████████| 80/80 [21:57<00:00, 16.47s/it]


In [62]:
#if __name__ == '__main__':
  #  with ElapsedTimer('VideoCaptioningPreProcessing'):
     #   fire.Fire(VideoCaptioningPreProcessing)